In [1]:
import pandas as pd

In [2]:
inpath = './presid_raw/'
outpath = './presid_clean/presid_clean.csv'

In [3]:
#load
df = {}
years = [1988, 1995, 2002, 2007, 2012, 2017]
for year in years:
    df[year] = pd.read_csv(inpath + '%d.csv' % year)

In [4]:
#réorganisation des colonnes
for year in years:
    df[year] = df[year][['Candidat', 'Departement', 'Parti', 'Score']]

In [5]:
df[1988].head()

,Candidat,Departement,Parti,Score
0,François Mitterrand,ain,PS,"31,41%"
1,Raymond Barre,ain,UDF,"21,21%"
2,Jacques Chirac,ain,RPR,"19,12%"
3,Jean-Marie Le Pen,ain,FN,"16,09%"
4,André Lajoinie,ain,PCF,"4,43%"


In [6]:
#creation d'une df vide pour concaténer toutes les années
df2 = pd.DataFrame(columns=['year', 'Departement', 'Parti', 'Score'])

In [7]:
df2

,year,Departement,Parti,Score


In [8]:
#concatenation des df
for year in years:
    df2 = df2.append(pd.DataFrame({'year': year,
                     'Departement': df[year].Departement,
                     'Parti': df[year].Parti,
                     'Score': df[year].Score}), ignore_index=True)

In [9]:
#test de comparaison avec les csv d'origine
df2[(df2.year == 1995) & (df2.Departement == "alpes-maritimes")]

,Departement,Parti,Score,year
990,alpes-maritimes,rpr,"22,48%",1995.0
991,alpes-maritimes,Front national,"22,48%",1995.0
992,alpes-maritimes,UDF,"20,75%",1995.0
993,alpes-maritimes,Parti socialiste,"16,36%",1995.0
994,alpes-maritimes,Parti communiste,"6,79%",1995.0
995,alpes-maritimes,\n\t\t\t\t\t\t\t\tMPF\t\t\t\t\t\t\t\t,"4,51%",1995.0
996,alpes-maritimes,Lutte ouvrière,"3,70%",1995.0
997,alpes-maritimes,Les Verts,"2,72%",1995.0
998,alpes-maritimes,\n\t\t\t\t\t\t\t\tNouvelle solidarité\t\t\t\t\...,"0,21%",1995.0


In [10]:
#maintenant on doit cleaner les partis
df2.Parti.unique()

array(['PS', 'UDF', 'RPR', 'FN', 'PCF', 'Les Verts',
       '\n\t\t\t\t\t\t\t\tCommuniste r\xc3\xa9novateur\t\t\t\t\t\t\t\t',
       'LO',
       '\n\t\t\t\t\t\t\t\tMouvement pour un parti des travailleurs\t\t\t\t\t\t\t\t',
       'Parti socialiste', 'Front national', 'rpr', 'Parti communiste',
       '\n\t\t\t\t\t\t\t\tMPF\t\t\t\t\t\t\t\t', 'Lutte ouvri\xc3\xa8re',
       '\n\t\t\t\t\t\t\t\tNouvelle solidarit\xc3\xa9\t\t\t\t\t\t\t\t',
       '\n\t\t\t\t\t\t\t\tParti ouvrier europ\xc3\xa9en\t\t\t\t\t\t\t\t',
       'MDC', 'DL', 'Les  Verts', 'CPNT', 'LCR', 'MNR', 'CAP 21', 'PRG',
       'FRS', 'PT', 'UMP', 'MPF', 'DIV', 'MoDem', 'Front de gauche',
       'Europe \xc3\xa9cologie - Les Verts', 'Debout la R\xc3\xa9publique',
       'NPA', 'Solidarit\xc3\xa9 et progr\xc3\xa8s', 'UPR',
       'R\xc3\xa9sistons'], dtype=object)

In [11]:
#un peu de clean
df2.Parti = df2.Parti.apply(lambda x: x.replace('\n\t\t\t\t\t\t\t\t', ''))

In [12]:
df2.Parti = df2.Parti.apply(lambda x: x.replace('\t\t\t\t\t\t\t\t', ''))

In [13]:
#dico contenant pour chaque parti le groupe auquel nous l'assignons
groups = {'PS': 'G', 'UDF': 'C', 'RPR': 'D', 'FN': 'ED',
         'PCF': 'EG', 'Les Verts': 'EG', 'Communiste rénovateur': 'EG', 
        'LO': 'EG', 'Mouvement pour un parti des travailleurs': 'EG',
         'Parti socialiste': 'G', 'Front national': 'ED', 'rpr': 'D',
         'Parti communiste': 'EG', 'MPF': 'ED', 'Lutte ouvrière': 'EG',
         'Nouvelle solidarité': 'Autre', 'Parti ouvrier européen': 'EG',
         'MDC': 'Autre', 'DL': 'D', 'Les  Verts': 'EG', 'CPNT': 'ED',
          'LCR': 'EG', 'MNR': 'ED', 'CAP 21': 'C', 'PRG': 'G', 'FRS': 'D',
          'PT': 'EG', 'UMP': 'D', 'DIV': 'EG', 'MoDem': 'C',
          'Front de gauche': 'EG', 'Europe écologie - Les Verts': 'EG',
          'Debout la République': 'ED', 'NPA': 'EG',
          'Solidarité et progrès': 'Autre', 'UPR': 'Autre', 'Résistons': 'Autre'}

In [14]:
#application du dico
df2.Parti = df2.Parti.apply(lambda x: groups[x])

In [15]:
df2.Parti.unique()

array(['G', 'C', 'D', 'ED', 'EG', 'Autre'], dtype=object)

In [16]:
#Arranger les scores
df2.Score = df2.Score.apply(lambda x: x.replace(',', '.').replace('%', ''))

In [20]:
df2.Score = df2.Score.apply(float)

In [21]:
#OK maintenant on aggrège
df2.head(10)

,Departement,Parti,Score,year
0,ain,G,31.41,1988.0
1,ain,C,21.21,1988.0
2,ain,D,19.12,1988.0
3,ain,ED,16.09,1988.0
4,ain,EG,4.43,1988.0
5,ain,EG,4.05,1988.0
6,ain,EG,1.82,1988.0
7,ain,EG,1.51,1988.0
8,ain,EG,0.37,1988.0
9,aisne,G,39.55,1988.0


In [23]:
#Allez, on change de nom de df pour l'occasion
df3 = df2.groupby(['year', 'Departement', 'Parti']).sum()

In [29]:
#Petit test v/s les csv d'origine
df3.loc[1995.0,'maine-et-loire',:]

Score
year   Departement    Parti       
1995.0 maine-et-loire Autre   0.30
                      C      25.08
                      D      20.03
                      ED     19.07
                      EG     14.60
                      G      20.92

In [31]:
#save
df3.to_csv(outpath)